In [219]:
'''
Name: Ankit Agrawal
Roll No: 194101006
Mtech, 2nd semester




I have implemented a CDCL based SAT solver,

Data Structure to store formulae: 2-literal watching (Lazy data structure)

Main Idea of sat solver:
backtracking + clause learning
With every decide and unit resolution I am maintaining a Implication Graph
When unit resolution detects a conflict, implemented algorithm will analyze the Implication Graph and
derive a conflict driven clause (learned clause).

Implemented a decide heuristic, description above "decide()" method

Backtracking level is decided by learned clause which is 2nd highest decision level in the learned clause,
I am backtracking to 2nd highest level so that after doing it learned clause becomes unit.
After backtracking learned clause is added to formulae and its watched literals are made to point to non-zero literals

When a conflict is detected without any decision, the formulae us marked UNSAT
When all the variables are assigned values and formulae remains SATISFIED, its marked SAT

------------------------------------------------------------------------------------

Code Organisation:
1. "Variable" Class: For each variable this class is created.
2. "Clause" Class: For each clause of formulae this class is created.
Variable Class and Clause Class maintain 2-watched literal data structure internally

3. "Formulae" Class: This class contains list of clauses which makes up the formulae.
4. "Implication_Graph" Class: Used to store implication graph.
5. "Sat_Solver" class: This class is the entry point of the program and is responsible to do the actual SAT solving

Detailed description of each class is commented above each class.

-------------------------------------------------------------------

To run the program give file name of .sat file to "main" method (at last)
It will output SAT/UNSAT
If you want to see variable assignments,
uncomment: "print(sat_solver.formulae.isAllVariablesAssigned())" inside main function

-------------------------------------------------------------------

Entry point of solver is main()
main() creates object of "Sat_Solver" class and give it file name.
Sat_solver init is responsible to created Formulae object

Input File is read by Formulae class and clauses are read and organised in 2-literal watching data structure.

'''

'\nI have implemented a CDCL based SAT solver,\nData Structure to store formulae: 2-literal watching\n\nMain Idea of sat solver:\nbacktracking + clause learning\nWith every decide and unit resolution I am maintaining a Implication Graph\nWhen unit resolution detects a conflict, implemented algorithm will analyze the Implication Graph and\nderive a conflict driven clause (learned clause).\n\nImplemented a decide heuristic, description above "decide()" method\n\nBacktracking level is decided by learned clause which is 2nd highest decision level in the learned clause,\nI am backtracking to 2nd highest level so that after doing it learned clause becomes unit.\nAfter backtracking learned clause is added to formulae and its watched literals are made to point to non-zero literals\n\nWhen a conflict is detected without any decision, the formulae us marked UNSAT\nWhen all the variables are assigned values and formulae remains SATISFIED, its marked SAT\n\n----------------------------------------

In [220]:
from itertools import islice
from enum import Enum

In [221]:
'''
Variable class represents each variable
'''
class Variable:
    def __init__(self, name, value = None):
        self.name = name #Name of variable
        self.value = value
        self.un_used = False #True if variable is used somewhere in clause
        if value is not None:
            self.un_used = True
        #positive and negative watched lists according to 2-literal data structure
        #Contains list of clause indices whose watched literal is pointing to positive literals/ negative literals
        self.positive_watched = list()
        self.negative_watched = list()

    #When a new clause is read/added updating watched lists
    def addClause(self, literal, clause_number):
        if literal > 0:
            self.positive_watched.append(clause_number)
        elif literal < 0:
            self.negative_watched.append(clause_number)
    
    '''
    Assigns value to variable
    Also if value assigned is 1, returns "self.negative_watched" list
    because watch pointers of clauses in above list have to point to new non zero literals
    '''
    def assign(self, value):
        self.value = value
        #clause_to_update is used when we are assigning variable zero value
        clause_to_update = None
        if value == 0:
            clause_to_update = self.positive_watched
        elif value == 1:
            clause_to_update = self.negative_watched
        else:
            pass
        return clause_to_update

        
    #For debugging
    def printVariable(self):
        if self.un_used:
            return
        print("Variable Name: \t\t", self.name)
        print("Variable Value: \t", self.value)
        print("Negative Watched: \t", self.negative_watched)
        print("Positive Watched: \t", self.positive_watched)
        print("*" * 20)

In [222]:
'''
Useful enums to denote clause status at any time, when ever any change happens to a clause
Its status is updated and is represented by below enums
'''
class Clause_Status(Enum):
    UNIT = 1 #All but one literal is assigned value 0
    SATISFIED = 2 #At least one literal is assigned value 1
    UNSATISFIED = 3 #All literals are assigned value 0
    UNRESOLVED = 4 #Otherwise
'''
Object of this class represents 1 clause
'''
class Clause:
    '''
    Literals is list of literals, which is read from file
    variables is variable list, whose length is number of variables
    Clause_numbering is index of current clause in Clauses list (Stored by formulae object)

    backtrack_list: is useful when a new learned clause is added to formulae in middle of program execution
    Above is none and not useful when a clause is read from file initially
    '''
    def __init__(self, literals, variables, clause_numbering, backtrack_list = None):
        self.literals = literals #List of literals, can contain negated literals
        self.literal1 = 0 #This is first watched literal, initially make it point to 0th literal
        self.literal2 = len(literals) - 1 #This is first watched literal, initially make it point to last literal
        self.status = Clause_Status.UNRESOLVED #This is the status when clause is created

        #Useful for learned clauses
        if backtrack_list:
            self.point_watched_literals(clause_numbering, variables, backtrack_list)

        #Now make sure clause is pointed variables watched list as required
        self.populateVariables(literals[self.literal1], literals[self.literal2], variables, clause_numbering)
    '''
    When a new clause id read from file, we need to inform lists of variables of clause
    This function adds information about clause to variable list
    Tells a variable that it is being used at this clause
    '''
    def populateVariables(self, literal1, literal2, variables, clause_numbering):

        #print("Populate: ", literal1, literal2, variables)
        pos_literal1 = abs(literal1)
        if literal2 is not None:
            pos_literal2 = abs(literal2)
        if literal1 != literal2 and literal2 is not None:
            #Both the watched literals are pointing to different variables
            variables[pos_literal1].addClause(literal1, clause_numbering)
            variables[pos_literal2].addClause(literal2, clause_numbering)
        else:
            #Both the watched literals are pointing to same variables
            variables[pos_literal1].addClause(literal1, clause_numbering)
            if literal2 is None:
                pass#print("here")
            if literal2 is not None:
                self.status = Clause_Status.UNIT
    #Traverse the clause literals list and try to find 2 non zero literals if possible
    #Returns index of such literal as list
    def get_two_non_zero_literals(self, variables):
        non_zero_index = []
        self.status = Clause_Status.UNRESOLVED
        for literal_index in range(len(self.literals)):
            literal = self.literals[literal_index]
            literal_value = self.get_literal_value(literal, variables)
            if literal_value != 0:
                non_zero_index.append(literal_index)
                if literal_value == 1:
                    self.status = Clause_Status.SATISFIED
                if len(non_zero_index) == 2:
                    break
        return non_zero_index
    '''
    When a new clause is learned, below method make sure watched literals of new clause points to
    proper literals.

    Watch literal with zero value will point to most recently assigned variable (variable assigned in highest decision level)
    Other watch literal will point to non zero value
    '''
    def point_watched_literals(self, clause_numbering, variables, backtrack_list):
        #print("In point_watched_literals")
        #print(self.printClause())
        #print(backtrack_list)
        #New clause should always be unit
        non_zero_literal_index = self.get_two_non_zero_literals(variables)
        length = len(non_zero_literal_index)
        #print("non_zero_literal_index", non_zero_literal_index)
        if len(self.literals) < 1:
            return
        if length == 2:
            self.literal1 = non_zero_literal_index[0]
            self.literal2 = non_zero_literal_index[1]
        elif length == 1:
            #Get variable assigned at highest level which is not equal to literal at non_zero index
            #print("non_zero_literal_index", non_zero_literal_index)
            self.literal1 = non_zero_literal_index[0]
            literal1_varname = abs(self.literals[self.literal1])
            literal2_name = None
            for bt_list_index in range(len(backtrack_list)-1, -1, -1):
                #print(bt_list_index)
                bt_varname, dl = backtrack_list[bt_list_index]
                neg = -1*bt_varname
                if bt_varname in self.literals:
                    if literal1_varname != bt_varname:
                        literal2_name = bt_varname
                        break
                elif neg in self.literals:
                    if neg != literal1_varname:
                        literal2_name = neg
                        break
            if literal2_name:
                #print("Pointing")
                self.literal2 = self.literals.index(literal2_name)
            else:
                print("Should not be possible")
                exit(1)
            #Will have to handle the case where total literals  = 1 in clause
        else:
            pass #Length is 0
        #self.printClause()
        return self.get_status(variables)
    '''
    Update and return status of clause
    '''
    def get_status(self, variables):
        l1_value = self.get_literal_value(self.literals[self.literal1], variables)
        l2_value = self.get_literal_value(self.literals[self.literal2], variables)

        if l1_value==0 and l2_value==0:
            self.status = Clause_Status.UNSATISFIED
        elif l1_value==1 or l2_value==1:
            self.status = Clause_Status.SATISFIED
        elif l1_value is None and l2_value is None and self.literal1 != self.literal2:
            self.status = Clause_Status.UNRESOLVED
        else:
            #One is None and other is zero
            self.status = Clause_Status.UNIT
        return self.status

    #This function corrects one of the watched literal pointer, if it is pointing to zero valued literal
    #After a variable is assigned value, we call update clause to reflect values in that clause
    def update_clause(self, literal_name, variables, clause_number):
        is_remove_from_list = False
        non_zero_index = self.get_two_non_zero_literals(variables)
        if len(non_zero_index) == 2:
            #self.status = Clause_Status.UNRESOLVED
            if abs(self.literals[self.literal1]) == literal_name:
                #Literal1 has to point to new position
                if non_zero_index[0] != self.literal2:
                    self.literal1 = non_zero_index[0]
                else:
                    self.literal1 = non_zero_index[1]
                #print("Populating")
                self.populateVariables(self.literals[self.literal1], None, variables, clause_number)
            else:
                #literal2 has to point to new position
                if non_zero_index[0] != self.literal1:
                    self.literal2 = non_zero_index[0]
                else:
                    self.literal2 = non_zero_index[1]
                #print("Populating")
                self.populateVariables(self.literals[self.literal2], None, variables, clause_number)
            is_remove_from_list = True
        elif len(non_zero_index) == 1:
            if self.status != Clause_Status.SATISFIED:
                self.status = Clause_Status.UNIT
        else:
            #All the literals have taken value 0
            self.status = Clause_Status.UNSATISFIED #Its unsatisfied clause
        return self.status, is_remove_from_list

    #For a literal in clause gets its value
    def get_literal_value(self, literal, variables):
        variable_index = abs(literal)
        variable_value = variables[variable_index].value
        if variable_value is None:
            return variable_value
        elif literal > 0:
            #Its a non negated literal, return value as it is
            return variable_value
        else:
            #This is a negated literal, its variable value should be negated
            return variable_value^1
        
    '''
    When a clause is detected to be unit, this method is called to get name and value of literal which has become unit
    and
    Return value which we have to assign to the unit clause's unassigned variable in order to satisfy it
    Also returns the variable name 
    '''
    def get_varname_value(self, variables):
        #self.printClause()
        #if self.status == Clause_Status.UNIT:
        l1 = self.literals[self.literal1]
        l2 = self.literals[self.literal2]
        #print("literal: ", l2)
        l1_value = self.get_literal_value(l1, variables)
        l2_value = self.get_literal_value(l2, variables)
        #print(variables[6].value)
        non_zero = self.get_two_non_zero_literals(variables)

        if l1_value == 1 or l2_value == 1:
            #print("Clause is SAT. Nothing to be done")
            self.status = Clause_Status.SATISFIED
            return None, self.status
        elif l1_value is None and l2_value is None and l1_value != l2_value:
            self.status = Clause_Status.UNRESOLVED
            return None, self.status
        elif len(non_zero) >=2:
            self.status = Clause_Status.UNRESOLVED
            return None, self.status
        elif l1_value is None :
            if l1 > 0:
                value = 1
            else:
                value = 0
            return (abs(l1), value), Clause_Status.UNIT
        elif l2_value is None :
            if l2 > 0:
                value = 1
            else:
                value = 0
            return (abs(l2), value), Clause_Status.UNIT
        else:
            #print("Clause has become UNSAT", non_zero, self.literal1, self.literal2, l1_value, l2_value)
            #if len(non_zero) == 1:
             #   print(self.get_literal_value(self.literals[non_zero[0]], variables))
            self.status = Clause_Status.UNSATISFIED
            return None, self.status
                #exit(1)
        # else:
        #     print("Clause is not unit.")
        #     return None


    #For Debugging
    def is_satisfied(self, variables):
        for l in self.literals:
            l_value = self.get_literal_value(l, variables)
            if l_value == 1:
                return True
        return False
    def printClause(self):
        print("Clause: \t", self.literals)
        print("Status: \t", self.status)
        print("Literal1: \t", self.literal1)
        print("Literal2: \t", self.literal2)
        print("*" * 20)
        


In [223]:
'''
This class is responsible to store complete formulae
To do so it takes help of Variable and Clause classes
'''
class Formulae:
    '''
    .cnf file name is provided to init method so that it can read from the file and initilize itself
    '''
    def __init__(self, filename):
        self.variables = list() #Create empty list to store variables, this list will be filled by object of variable class
        self.total_variables = -1
        self.total_clauses = 0
        self.clauses = list() #Empty list to store clauses, later filled by objects of Clause class
        self.nos_assigned_vars = 0 #Used to tell at any instant how many variables are assigned value
        self.list_of_unit_clauses = list() #This list keeps track of unit clauses, to ease unit resolution process
        #Read formulae from file
        self.readFormulae(filename)
        
    '''
    Reads clause line by line and populates clause list
    '''
    def readFormulae(self, fileName):
        #print("Trying to open file")
        with open(fileName) as fp:
            line  = fp.readline()
            while line:
                line = line.strip()
                if line[0] == "p":
                    a, b, c, d = line.split()
                    self.total_variables = int(c)
                    #self.total_clauses = int(d)
                    self.variables = [None] * (self.total_variables + 1)
                elif line[0] == "c":
                    #Its a comment
                    pass
                else:
                    #Its a clause
                    line = line.split()[:-1]
                    clause_literals = [int(i) for i in line]
                    #Create a Clause object and store it in list
                    self.add_clause(clause_literals)

                line = fp.readline()
        for var_index in range(1, len(self.variables)):
            #If some variable does not show up in formulae we assgn it value 1
            if self.variables[var_index] is None:
                self.variables[var_index]  = Variable(var_index, 1)
                #print("here\n\n\n\n")
                self.nos_assigned_vars += 1
    '''
    Creates a clause object and store it
    '''
    def add_clause(self, clause_literals, variable_assignments = None):
        #Check if valid literals are received
        self.check_literals(clause_literals)
        self.total_clauses += 1
        index = len(self.clauses)
        #Create clause object
        clause = Clause(clause_literals, self.variables, index, variable_assignments)
        self.clauses.append(clause)
        if clause.status == Clause_Status.UNIT:
            self.list_of_unit_clauses.append(index)

    '''
    When new clause is added, this method checks if variables of literals are not out of range
    If .cnf file says number of variables = x
    Then all literals should be in range "[-x, x] - {0}"
    '''
    def check_literals(self, literals):
        negTotal = -1 * self.total_variables
        for l in literals:
            posL = max(l, -1 * l)
            if l > self.total_variables or l < negTotal:
                print("Out of range clause received. Invalid Input")
                exit(1)
            elif self.variables[posL] is None:
                self.variables[posL] = Variable(posL)

    '''
    When either because of decide or uint propogation a variable needs to be assgned value
    Below method is called
    '''
    def assign_variable(self, var_name, value):
        #Get list of clauses which are to be updated because, due to assigning value to variable some watch literals will point to zero values
        #We have to make them point to non_zero as per 2-watched literal data structure
        clause_to_update = self.variables[var_name].assign(value)
        self.nos_assigned_vars += 1
        #print("self.nos_assigned_vars += 1", self.nos_assigned_vars)
        for clause_index in clause_to_update[:]:
            status, remove = self.clauses[clause_index].update_clause(var_name, self.variables, clause_index)
            if remove:
                #Moving the watched literal operation

                #Now our clause is no longer pointing to variable
                #Remove clause entry from variable
                #print("Removing. VarName:", var_name, "Clause Index:", clause_index)
                clause_to_update.remove(clause_index)
            elif status == Clause_Status.UNIT or status == Clause_Status.UNSATISFIED:
                self.list_of_unit_clauses.append(clause_index)
    '''
    Called during back propogation
    When we want to unassign value of a variable
    '''
    def unassign_variable(self, var_name):
        self.nos_assigned_vars -= 1
        self.variables[var_name].assign(None)
    '''
    Called during unit resolution phase while solving SAT
    Its purpose is to assign such a value to the variable so that value of unit clause becomes 1 
    '''
    def unit_resolve_clause(self, clause_index):
        #print("Trying to unit resolve clause index:", clause_index)
        unit_clause = self.clauses[clause_index]
        #print("Before get varname variable, clauseIndex: ", clause_index)
        rv, status = unit_clause.get_varname_value(self.variables)
        #print(rv, status)
        if status == Clause_Status.UNIT:
            var_name, value = rv
            #print("Var Name: ", var_name, "Value: ", value)
            self.assign_variable(var_name, value)
            #print("After assign variable")
            return status, rv
        elif status == Clause_Status.SATISFIED:
            return status, None
        elif status == Clause_Status.UNSATISFIED:
            return status, None
        elif status == Clause_Status.UNRESOLVED:
            return status, None

    #Decides termination of sat solver
    def isAllVariablesAssigned(self):
        if self.nos_assigned_vars == self.total_variables:
            return True
        else:
            return False

    #For Debugging
    '''
    After run, find clause which has not been satisfied
    '''
    def check_satisfaction(self):
        for clause in self.clauses:
            if not clause.is_satisfied(self.variables):
                print("Clause Not Satisfied: ")
                clause.printClause()
                break
        else:
            print("Clause is satisfied")

    def printClauses(self):       
        for i in range(len(self.clauses)):
            print("Clause Number: \t", i)
            self.clauses[i].printClause()
        print("-"*50)

    def printVariables(self):
        for i in range(1, self.total_variables + 1):
            if self.variables[i] is not None:
                #print("Variable Number: \t", i)
                self.variables[i].printVariable()
        print("-"*50)

In [224]:
'''
Stores implication graph data structure, which is easy to learn clause according to handbook's eqh 4.18
'''
class Implication_Graph:
    def __init__(self):
        self.g = {}
        #self.current_dl = 0
    def add_vertex(self, var_name, value, decision_level, antecedent_clause_index):
        if var_name in self.g:
            print("Var name already present in graph")
            exit(1)
        self.g[var_name] = {"value": value,
                            "dl": decision_level,
                            "antecedent": antecedent_clause_index}

    def print_graph(self):
        print(self.g)

In [225]:
#Overall formula status
class Formulae_Status(Enum):
    SAT = 1
    UNSAT = 2
    UNKNOWN = 3

'''
It is the main sat solver class which solves the problem
'''
class Sat_Solver:
    def __init__(self, fileName):
        #Read and initialize formulae
        self.formulae = Formulae(fileName)
        #Initialize inplication graph
        self.implication_graph = Implication_Graph()
        self.variable_assignment = list() #Used for backtracking

    '''
    This is the CDCL implementation which is called to solve the problem
    '''
    def solve(self):
        decision_level = -1
        #print(self.formulae.list_of_unit_clauses)
        #Try to resolve unit clauses
        if self.unit_resolution(decision_level) == Clause_Status.UNSATISFIED:
            print("Hre")
            return Formulae_Status.UNSAT
        #decision_level += 1

        while not self.formulae.isAllVariablesAssigned():
            #If all variables is not assigned make a decision
            decision_level += 1
            self.decide(decision_level)
            while self.unit_resolution(decision_level) == Clause_Status.UNSATISFIED:

                #If there is a conflict and implication graph has no node means
                #Conflict without any decision
                if len(self.implication_graph.g) == 0:
                    return Formulae_Status.UNSAT
                    #Because we are getting conflict wothuot any decision

                new_clause_literals, decision_level = self.learn_clause(decision_level)
                if len(new_clause_literals) == 0:
                    #Not able to learn new clause, generally because all variables are at DL -1
                    return Formulae_Status.UNSAT
                else:
                    #print("BT to DL:", decision_level, "\n\n")
                    #Backtrack to DL given by learn clause
                    self.backtrack(decision_level)
                    #Add a new fFormulae
                    self.formulae.add_clause(new_clause_literals, self.variable_assignment)
        #Got out of while loop means all variable are assigned
        return Formulae_Status.SAT

    '''
    Recursively carries out unit resolution until there is no more unit clause
    If conflict is detected during unit resolution it reports it
    '''
    def unit_resolution(self, decision_level):
        #print("Unit Clauses: ", self.formulae.list_of_unit_clauses)
        while self.formulae.list_of_unit_clauses:
            #When list of unit clauses is not empty
            clause_index = self.formulae.list_of_unit_clauses.pop()
            status, name_value = self.formulae.unit_resolve_clause(clause_index)
            if status == Clause_Status.UNIT:
                #If the clause we resolved was really uint
                variable_name, value = name_value
                self.variable_assignment.append((variable_name, decision_level))
                self.implication_graph.add_vertex(variable_name, value, decision_level, clause_index)
            elif status == Clause_Status.UNSATISFIED:
                #If clause has become unsatisfied, we have detected a conflict
                self.implication_graph.add_vertex("k", None, decision_level, clause_index)
                #print("Formulae is becoming unsatisfied for clause index:", clause_index)
                return status
        return None

    '''
    This is the decide.
    Here we choose a variable using heuristic

    Such a variable is choosen which will make most of clauses pointed by watched literals of variable SATISFIED
    '''
    def decide(self, decision_level):
        decided_variable_name = None
        decided_variable_value = None
        max_clause_len = 0
        #print(self.formulae.variables)

        #Find a variable with largest watched literal class
        #Assign value to such variable accordingly

        for var in islice(self.formulae.variables, 1, None):
            #print(var)
            if var.value is None:
                #We got a potential candidate to assign value to
                if len(var.positive_watched) > max_clause_len:
                    max_clause_len = len(var.positive_watched)
                    decided_variable_name = var.name
                    decided_variable_value = 1
                if len(var.negative_watched) > max_clause_len:
                    max_clause_len = len(var.negative_watched)
                    decided_variable_name = var.name
                    decided_variable_value = 0

        #print("Deciding:", decided_variable_name, decided_variable_value, "Dl:", decision_level)
        self.variable_assignment.append((decided_variable_name, decision_level))
        self.formulae.assign_variable(decided_variable_name, decided_variable_value)
        self.implication_graph.add_vertex(decided_variable_name, decided_variable_value, decision_level, None)

    '''
    When a conflict is detected we backtrack to given level
    '''
    def backtrack(self, decision_level):
        #print("Backtracking List is: ", self.variable_assignment)
        #print("BT level", decision_level)
        del self.implication_graph.g["k"]

        #variable_assignment list stores the sequence in which variables were assgned values
        #We proceed in reverse order to backtrack

        while self.variable_assignment:
            var_name, dl = self.variable_assignment.pop()
            if dl > decision_level:
                del self.implication_graph.g[var_name]
                self.formulae.unassign_variable(var_name)
            else:
                self.variable_assignment.append((var_name, dl))
                break

    '''
    Performs Conflict_Analysis
    When ever a conflict is detected we try to learn a new clause
    This function will learn a new clause,
    Decide DL to backtrack to
    Calls backtrack function to backtrack to that DL
    Returns learned clause


    This is the implementation of equation: 4.18 of handbook
    '''
    def learn_clause(self, decision_level):
        #print("Learning @: ", decision_level)
        #print(self.implication_graph.g)
        resolvers = {"k"}
        learned_clause = set()
        highest_dl = -1
        second_highest_dl = -1
        nos_literals_dl = 0
        i = 0

        while resolvers:
            i += 1
            #Get a variable to resolve
            var_name = resolvers.pop()
            node_details = self.implication_graph.g[var_name]
            var_antecedent = self.formulae.clauses[node_details["antecedent"]].literals

            #Resolve operation on learned_clause and var_antecedent
            for literal in var_antecedent:
                variable = abs(literal)
                node_details = self.implication_graph.g[variable]
                variable_dl = node_details["dl"]
                if (-1*literal) in learned_clause:
                    learned_clause.remove(-1*literal)
                    if variable_dl == decision_level:
                        #print("Subtracting: ", literal, nos_literals_dl)
                        nos_literals_dl -= 1
                else:
                    learned_clause.add(literal)
                    if variable_dl == decision_level:
                        nos_literals_dl += 1
                        if node_details["antecedent"] is not None:
                            #print(type(resolvers))
                            resolvers.add(variable)

            #Resolution step is complete, now check for termination condition
            #Termination condition of UIP
            if nos_literals_dl == 1 and i != 1:
                #print("BREAKING FROM HERE")
                #print("Learned: ", learned_clause, "nos_dl", nos_literals_dl, "2nd H dL", second_highest_dl, "resolvers: ", resolvers)
                break

            #print("Learned: ", learned_clause, "nos_dl", nos_literals_dl, "2nd H dL", second_highest_dl, "resolvers: ", resolvers)
            #return None, None

        #We have learned the clause, now compute backtracking level
        #Backtracking level is 2nd highest level in learned clause
        for literal in learned_clause:
            variable_dl = self.implication_graph.g[abs(literal)]["dl"]
            #Update second highest DL for backtracking
            if variable_dl > highest_dl:
                second_highest_dl = highest_dl
                highest_dl = variable_dl
            elif variable_dl != highest_dl and variable_dl > second_highest_dl:
                second_highest_dl = variable_dl
        #print("FINAL   Learned: ", learned_clause, "nos_dl", nos_literals_dl, "2nd H dL", second_highest_dl, "resolvers: ", resolvers)
        learned_clause = list(learned_clause)
        return learned_clause, second_highest_dl

    #Below methods for debugging puropse
    def check_clause_assignment(self):
        self.formulae.check_satisfaction()
    def test(self):
        print("SOLVE ANS: ", self.solve())

    def print_things(self):
        print("Unit Clauses: ", self.formulae.list_of_unit_clauses)
        print("-" * 50)
        self.implication_graph.print_graph()
        print("-" * 50)
        self.formulae.printClauses()
        self.formulae.printVariables()
            

In [226]:
#Its is the main method which is responsible for overall control of program
def main(filename):

    sat_solver = Sat_Solver(filename)
    status = sat_solver.solve()
    print("\n\n\n\nFormulae_Status.SAT: Means formula is satisfied\nFormulae_Status.UNSAT: Means Formula is not satisfied")
    print("\nSOLVE ANS: ", status)
    #sat_solver.print_things()
    #try:
    #sat_solver.test()
    #except:

    #sat_solver.print_things()

    #print(sat_solver.formulae.isAllVariablesAssigned())

In [227]:
#call main with file name
main("unsat3.cnf")





Formulae_Status.SAT: Means formula is satisfied
Formulae_Status.UNSAT: Means Formula is not satisfied

SOLVE ANS:  Formulae_Status.SAT
